In [1]:
# 导入包
import numpy as np
import pandas as pd
import os, json
from pyecharts.charts import Line, Pie, Bar, Grid, Geo, Map
import pyecharts.options as opts
from pyecharts.globals import ChartType, SymbolType, ThemeType # SymbolType标点的样式


# 解决pyecharts显示空白
from pyecharts.globals import CurrentConfig, NotebookType
CurrentConfig.NOTEBOOK_TYPE = NotebookType.JUPYTER_NOTEBOOK

In [2]:
pd.set_option('display.max_rows', None) # 显示所有行
pd.set_option('display.max_columns', None) # 显示所有列
# 文件根路径
filepath_cn = "./疫情数据/国内数据/"
filepath_fg = "./疫情数据/外国数据/"
filepath_visual = './部分可视化页面/'
if not os.path.exists(filepath_visual): # 先判断这个文件夹是否已存在\
        os.mkdir(filepath_visual)

In [3]:
# 格式化时间 传入pandas 的一列
def format_time(time_series):
    # 格式化时间
    time_series.drop_duplicates('日期', inplace=True, ignore_index=True) # 先去除重复的列
    start_time = str(time_series['日期'][0])
    if start_time[-2] == '.':
        start_time += '0' # 因为float型转为字符串1,2就少个0，应该是1.20，因此先把0加上
    start_time = start_time.replace('.', '/') + '/2020'
    # print(start_time)
    end_time = str(time_series['日期'][len(time_series['日期'])-1])
    if end_time[-2] == '.':
        end_time += '0'
    end_time = end_time.replace('.', '/') + '/2020'
    # print(end_time)
    time_series['日期'] = pd.date_range(start=start_time, end=end_time)
    time_series['日期'] = time_series['日期'].astype(str)
    # cn_prov_day_data['日期']

# 外国疫情地图可视化

## 外国整体数据可视化

In [4]:
pieces=[
    {"max": 4000000, "min": 1000000, "label": ">1000000", "color": "#C71585"},
    {"max": 999999, "min": 100000, "label": "100000-999999", "color": "#B40404"},
    {"max": 99999, "min": 10000, "label": "10000-99999", "color": "#DF0101"},
    {"max": 9999, "min": 1000, "label": "1000-9999", "color": "#F78181"},
    {"max": 999, "min": 500, "label": "500-999", "color": "#F5A9A9"},
    {"max": 499, "min": 0, "label": "0-499", "color": "#AAAAAA"},
]
# 颜色块

In [5]:
nameMap = {
        'Afghanistan':'阿富汗','Albania':'阿尔巴尼亚','Algeria':'阿尔及利亚', 'Andorra':'安道尔','Angola':'安哥拉', 'Antarctica':'南极洲',
        'Antigua and Barbuda':'安提瓜和巴布达','Argentina':'阿根廷','Armenia':'亚美尼亚','Australia':'澳大利亚','Austria':'奥地利',
        'Azerbaijan':'阿塞拜疆','The Bahamas':'巴哈马','Bahrain':'巴林','Bangladesh':'孟加拉','Barbados':'巴巴多斯','Belarus':'白俄罗斯',
        'Belgium':'比利时','Belize':'伯利兹','Benin':'贝宁','Bermuda':'百慕大','Bhutan':'不丹','Bolivia':'玻利维亚','Bosnia and Herzegovina':'波黑',
        'Botswana':'博茨瓦纳','Brazil':'巴西','Brunei':'文莱','Bulgaria':'保加利亚','Burkina Faso':'布基纳法索','Burundi':'布隆迪','Cambodia':'柬埔寨',
        'Cameroon':'喀麦隆','Canada':'加拿大','Cape Verde':'佛得角','Central African Rep.':'中非共和国','Chad':'乍得','Chile':'智利','China':'中国',
        'Colombia':'哥伦比亚','Comoros':'科摩罗','Republic of the Congo':'刚果（布）','Costa Rica':'哥斯达黎加','Croatia':'克罗地亚','Cuba':'古巴',
        'Cyprus':'塞浦路斯','Czech Republic':'捷克','Djibouti':'吉布提','The Commonwealth of Dominica':'多米尼克','Dominican Republic':'多米尼加',
        'Ecuador':'厄瓜多尔','Egypt':'埃及','El Salvador':'萨尔瓦多','Equatorial Guinea':'赤道几内亚','Eritrea':'厄立特里亚','Estonia':'爱沙尼亚',
        'Ethiopia':'埃塞俄比亚','Falkland Islands':'福克兰群岛','Faroe Islands':'法罗群岛','Fiji':'斐济','Finland':'芬兰','France':'法国','French Guiana':'法属圭亚那',
        'French Southern and Antarctic Lands':'法属南半球和南极领地','Gabon':'加蓬','Gambia':'冈比亚','Gaza Strip':'巴勒斯坦','Georgia':'格鲁吉亚',
        'Germany':'德国','Ghana':'加纳','Greece':'希腊','Greenland':'丹麦','Grenada':'格林纳达','Guadeloupe':'瓜德罗普','Guatemala':'危地马拉',
        'Guinea':'几内亚','Guinea Bissau':'几内亚比绍','Guyana':'圭亚那','Haiti':'海地','Honduras':'洪都拉斯','Hong Kong':'香港','Hungary':'匈牙利',
        'Iceland':'冰岛','India':'印度','Indonesia':'印度尼西亚','Iran':'伊朗','Iraq':'伊拉克','Iraq-Saudi Arabia Neutral Zone':'伊拉克阿拉伯中立区',
        'Ireland':'爱尔兰','Isle of Man':'马恩岛','Israel':'以色列','Italy':'意大利',"Côte d'Ivoire":"科特迪瓦",'Jamaica':'牙买加','Jan Mayen':'扬马延岛',
        'Japan':'日本本土','Jordan':'约旦','Kazakhstan':'哈萨克斯坦','Kenya':'肯尼亚','Kerguelen':'凯尔盖朗群岛','Kiribati':'基里巴斯','North Korea':'北朝鲜',
        'South Korea':'韩国','Kuwait':'科威特','Kyrgyzstan':'吉尔吉斯斯坦','Laos':'老挝','Latvia':'拉脱维亚','Lebanon':'黎巴嫩','Lesotho':'莱索托',
        'Liberia':'利比里亚','Libya':'利比亚','Liechtenstein':'列支敦士登公国','Lithuania':'立陶宛','Luxembourg':'卢森堡','Macau':'澳门','Macedonia':'北马其顿',
        'Madagascar':'马达加斯加','Malawi':'马拉维','Malaysia':'马来西亚','Maldives':'马尔代夫','Mali':'马里','Malta':'马耳他','Martinique':'马提尼克',
        'Mauritania':'毛里塔尼亚','Mauritius':'毛里求斯','Mexico':'墨西哥','Moldova':'摩尔多瓦','Monaco':'摩纳哥','Mongolia':'蒙古','Morocco':'摩洛哥',
        'Mozambique':'莫桑比克','Myanmar':'缅甸','Namibia':'纳米比亚','Nepal':'尼泊尔','Netherlands':'荷兰','New Caledonia':'新喀里多尼亚','New Zealand':'新西兰',
        'Nicaragua':'尼加拉瓜','Niger':'尼日尔','Nigeria':'尼日利亚','Northern Mariana Islands':'北马里亚纳群岛','Norway':'挪威','Oman':'阿曼',
        'Pakistan':'巴基斯坦','Panama':'巴拿马','Papua New Guinea':'巴布亚新几内亚','Paraguay':'巴拉圭','Peru':'秘鲁','Philippines':'菲律宾',
        'Poland':'波兰','Portugal':'葡萄牙','Puerto Rico':'波多黎各','Qatar':'卡塔尔','Reunion':'留尼旺岛','Romania':'罗马尼亚','Russia':'俄罗斯',
        'Rwanda':'卢旺达','San Marino':'圣马力诺','Sao Tome and Principe':'圣多美和普林西比','Saudi Arabia':'沙特阿拉伯', 'Senegal':'塞内加尔',
        'Seychelles':'塞舌尔','Sierra Leone':'塞拉利昂','Singapore':'新加坡','Slovakia':'斯洛伐克','Slovenia':'斯洛文尼亚','Solomon Islands':'所罗门群岛',
        'Somalia':'索马里','South Africa':'南非','Spain':'西班牙','Sri Lanka':'斯里兰卡','St. Christopher-Nevis':'圣','St. Lucia':'圣卢西亚',
        'St. Vincent and the Grenadines':'圣文森特和格林纳丁斯','Sudan':'苏丹','Suriname':'苏里南','Svalbard':'斯瓦尔巴特群岛','Swaziland':'斯威士兰',
        'Sweden':'瑞典','Switzerland':'瑞士','Syria':'叙利亚','Taiwan':'台湾','Tajikistan':'塔吉克斯坦','Tanzania':'坦桑尼亚',
        'Thailand':'泰国','Togo':'多哥','Tonga':'汤加','Trinidad and Tobago':'特里尼达和多巴哥','Tunisia':'突尼斯','Turkey':'土耳其','Turkmenistan':'土库曼斯坦',
        'Turks and Caicos Islands':'特克斯和凯科斯群岛','Uganda':'乌干达','Ukraine':'乌克兰','United Arab Emirates':'阿联酋','United Kingdom':'英国',
        'United States':'美国','Uruguay':'乌拉圭','Uzbekistan':'乌兹别克斯坦','Vanuatu':'瓦努阿图','Venezuela':'委内瑞拉','Vietnam':'越南',
        'Western Sahara':'西撒哈拉','Western Samoa':'西萨摩亚','Yemen':'也门','Yugoslavia':'南斯拉夫','Dem. Rep. Congo':'刚果（金）',
        'Zambia':'赞比亚','Zimbabwe':'津巴布韦','S. Sudan':'南苏丹','Somaliland':'索马里兰','Montenegro':'黑山','Kosovo':'科索沃','Republic of Serbia':'塞尔维亚',
        'The Federation of Saint Kitts and Nevis':'圣基茨和尼维斯'
        }

country_names = {v:k for k, v in nameMap.items()} # 键值对互换
country_names

{'阿富汗': 'Afghanistan',
 '阿尔巴尼亚': 'Albania',
 '阿尔及利亚': 'Algeria',
 '安道尔': 'Andorra',
 '安哥拉': 'Angola',
 '南极洲': 'Antarctica',
 '安提瓜和巴布达': 'Antigua and Barbuda',
 '阿根廷': 'Argentina',
 '亚美尼亚': 'Armenia',
 '澳大利亚': 'Australia',
 '奥地利': 'Austria',
 '阿塞拜疆': 'Azerbaijan',
 '巴哈马': 'The Bahamas',
 '巴林': 'Bahrain',
 '孟加拉': 'Bangladesh',
 '巴巴多斯': 'Barbados',
 '白俄罗斯': 'Belarus',
 '比利时': 'Belgium',
 '伯利兹': 'Belize',
 '贝宁': 'Benin',
 '百慕大': 'Bermuda',
 '不丹': 'Bhutan',
 '玻利维亚': 'Bolivia',
 '波黑': 'Bosnia and Herzegovina',
 '博茨瓦纳': 'Botswana',
 '巴西': 'Brazil',
 '文莱': 'Brunei',
 '保加利亚': 'Bulgaria',
 '布基纳法索': 'Burkina Faso',
 '布隆迪': 'Burundi',
 '柬埔寨': 'Cambodia',
 '喀麦隆': 'Cameroon',
 '加拿大': 'Canada',
 '佛得角': 'Cape Verde',
 '中非共和国': 'Central African Rep.',
 '乍得': 'Chad',
 '智利': 'Chile',
 '中国': 'China',
 '哥伦比亚': 'Colombia',
 '科摩罗': 'Comoros',
 '刚果（布）': 'Republic of the Congo',
 '哥斯达黎加': 'Costa Rica',
 '克罗地亚': 'Croatia',
 '古巴': 'Cuba',
 '塞浦路斯': 'Cyprus',
 '捷克': 'Czech Republic',
 '吉布提': 'Djibouti',
 '多米尼克': '

In [6]:
world_all_data = pd.read_csv(filepath_fg+"外国疫情数据（按国家或大洲查看）.csv", encoding='GBK')
day = world_all_data['date'][0]
world_all_data

,name,continent,date,isUpdated,confirmAdd,confirmAddCut,confirm,suspect,dead,heal,nowConfirm,confirmCompare,nowConfirmCompare,healCompare,deadCompare
0,美国,北美洲,7.22,True,67140,0,4028569,0,144953,1886583,1997033,67140,29427,36594,1119
1,巴西,南美洲,7.22,False,41008,0,2159654,0,81487,1563382,514785,41008,-2298,41939,1367
2,印度,亚洲,7.22,False,37724,0,1193078,0,28732,753050,411296,37724,8604,28472,648
3,俄罗斯,欧洲,7.22,False,5806,0,787846,0,12722,570984,204140,5806,-3942,9587,161
4,南非,非洲,7.22,False,8170,0,381798,0,5368,208144,168286,8170,-5304,13279,195
5,秘鲁,南美洲,7.22,False,8497,0,362087,0,13579,248746,99762,8497,4440,3665,392
6,墨西哥,北美洲,7.22,False,6859,0,356255,0,40400,267407,48448,6859,1628,4316,915
7,智利,南美洲,7.22,False,3753,0,334683,0,8677,306816,19190,3753,-1443,5022,174
8,西班牙,欧洲,7.22,True,1358,0,313274,0,28424,196958,87892,1358,1356,0,2
9,英国,欧洲,7.22,False,0,0,297389,0,45507,1414,250468,0,-1,1,0


In [7]:
# 读取中国数据
china_data = pd.read_csv(filepath_cn+"中国疫情数据.csv", encoding='GBK').loc[0, :]
china_data

name             中国
nowConfirm     1014
confirm       86340
suspect           1
dead           4655
deadRate       5.39
heal          80671
healRate      93.43
Name: 0, dtype: object

In [8]:
# 读取中国数据
china_data2 = pd.read_csv(filepath_cn+"国内历史每日新增数据.csv", encoding='GBK').iloc[-1, :]
china_data2

confirm         74.00
suspect          0.00
dead             2.00
heal            45.00
importedCase     5.00
infect          22.00
deadRate         2.70
healRate        60.80
date             7.21
Name: 180, dtype: float64

In [9]:
data_pair_list = [(country_names[i], int(j)) for i, j in zip(world_all_data['name'], world_all_data['confirm']) if i in country_names]
data_pair_list.append(('China', int(china_data['confirm'])))
(
    Map(init_opts=opts.InitOpts(theme=ThemeType.ROMANTIC, width='1600px', height='800px'))
    .add(
        maptype="world", # 使用世界地图
        series_name='累计确诊', 
        data_pair=data_pair_list, # data_pair是放数据的地方，类别和属性值
        label_opts=opts.LabelOpts(is_show=False),
        is_map_symbol_show=False,
    ) # 也可以隐藏地图上的文字显示，不过悬浮点上的时候还是可以显示的 
    .set_global_opts(
        title_opts=opts.TitleOpts(title=f"世界 {day} 累计确诊"),
        visualmap_opts=opts.VisualMapOpts(is_piecewise=True, pieces=pieces) # 视觉印象配置项 is_piecewise=True就不是连续的颜色了，而是拆分成块了
    )
).render(f"{filepath_visual}世界 {day} 累计确诊.html")

f:\python\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


'C:\\Users\\Jarvis\\Desktop\\Python实习\\PythonInternship\\Python爬虫学习\\部分可视化页面\\世界 7.22 累计确诊.html'

In [10]:
data_pair_list = [(country_names[i], int(j)) for i, j in zip(world_all_data['name'], world_all_data['confirmAdd']) if i in country_names]
data_pair_list.append(('China', int(china_data2['confirm'])))
(
    Map(init_opts=opts.InitOpts(theme=ThemeType.ROMANTIC, width='1600px', height='800px'))
    .add(
        maptype="world", # 使用世界地图
        series_name='新增确诊', 
        data_pair=data_pair_list, # data_pair是放数据的地方，类别和属性值
        label_opts=opts.LabelOpts(is_show=False),
        is_map_symbol_show=False,
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title=f"世界 {day} 新增确诊"),
        visualmap_opts=opts.VisualMapOpts(is_piecewise=True, pieces=pieces) # 使用自定义颜色范围
    )
).render(f"{filepath_visual}世界 {day} 新增确诊.html")

f:\python\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


'C:\\Users\\Jarvis\\Desktop\\Python实习\\PythonInternship\\Python爬虫学习\\部分可视化页面\\世界 7.22 新增确诊.html'

In [11]:
data_pair_list = [(country_names[i], int(j)) for i, j in zip(world_all_data['name'], world_all_data['dead']) if i in country_names]
data_pair_list.append(('China', int(china_data['dead'])))
(
    Map(init_opts=opts.InitOpts(theme=ThemeType.ROMANTIC, width='1600px', height='800px'))
    .add(
        maptype="world", # 使用世界地图
        series_name='累计死亡', 
        data_pair=data_pair_list, # data_pair是放数据的地方，类别和属性值
        label_opts=opts.LabelOpts(is_show=False),
        is_map_symbol_show=False,
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title=f"世界 {day} 累计死亡"),
        visualmap_opts=opts.VisualMapOpts(is_piecewise=True, pieces=pieces) # 使用自定义颜色范围
    )
).render(f"{filepath_visual}世界 {day} 累计死亡.html")

f:\python\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


'C:\\Users\\Jarvis\\Desktop\\Python实习\\PythonInternship\\Python爬虫学习\\部分可视化页面\\世界 7.22 累计死亡.html'

In [12]:
data_pair_list = [(country_names[i], int(j)) for i, j in zip(world_all_data['name'], world_all_data['heal']) if i in country_names]
data_pair_list.append(('China', int(china_data['heal'])))
(
    Map(init_opts=opts.InitOpts(theme=ThemeType.ROMANTIC, width='1600px', height='800px'))
    .add(
        maptype="world", # 使用世界地图
        series_name='累计治愈', 
        data_pair=data_pair_list, # data_pair是放数据的地方，类别和属性值
        label_opts=opts.LabelOpts(is_show=False),
        is_map_symbol_show=False,
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title=f"世界 {day} 累计治愈"),
        visualmap_opts=opts.VisualMapOpts(is_piecewise=True, pieces=pieces) # 使用自定义颜色范围
    )
).render(f"{filepath_visual}世界 {day} 累计治愈.html")

f:\python\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


'C:\\Users\\Jarvis\\Desktop\\Python实习\\PythonInternship\\Python爬虫学习\\部分可视化页面\\世界 7.22 累计治愈.html'

In [13]:
data_pair_list = [(country_names[i], int(j)) for i, j in zip(world_all_data['name'], world_all_data['nowConfirm']) if i in country_names]
data_pair_list.append(('China', int(china_data['nowConfirm'])))
(
    Map(init_opts=opts.InitOpts(theme=ThemeType.ROMANTIC, width='1600px', height='800px'))
    .add(
        maptype="world", # 使用世界地图
        series_name='现有确诊', 
        data_pair=data_pair_list, # data_pair是放数据的地方，类别和属性值
        label_opts=opts.LabelOpts(is_show=False),
        is_map_symbol_show=False,
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title=f"世界 {day} 现有确诊"),
        visualmap_opts=opts.VisualMapOpts(is_piecewise=True, pieces=pieces) # 使用自定义颜色范围
    )
).render(f"{filepath_visual}世界 {day} 现有确诊.html")

f:\python\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


'C:\\Users\\Jarvis\\Desktop\\Python实习\\PythonInternship\\Python爬虫学习\\部分可视化页面\\世界 7.22 现有确诊.html'

## 部分国家具体行政区可视化

In [14]:
from pyecharts.datasets import register_url
register_url("https://echarts-maps.github.io/echarts-countries-js/")

In [15]:
# 可选国家有：韩国、美国、日本（用输入日本本土）、伊朗、意大利、英国
country = input('请输入想查看具体行政区的国家：')
country_region_data = pd.read_csv(filepath_fg+f"部分国家具体行政区数据/{country}-具体行政区数据.csv", encoding='GBK')
day = country_region_data['date'][0]
if country == '日本本土':
    country = '日本'
country_region_data

请输入想查看具体行政区的国家：美国


,name,date,nameMap,isUpdated,confirmAdd,confirmAddCut,confirm,suspect,dead,heal
0,纽约,7.22,New York,True,0,0,413315,0,32520,58366
1,加利福尼亚,7.22,California,True,0,0,409305,0,7890,108711
2,佛罗里达,7.22,Florida,True,0,0,369834,0,5206,381
3,德克萨斯,7.22,Texas,True,0,0,351113,0,4213,186529
4,新泽西,7.22,New Jersey,True,0,0,177256,0,15826,3973
5,伊利诺伊,7.22,Illinois,True,0,0,163689,0,7324,75880
6,乔治亚,7.22,Georgia,True,0,0,148988,0,3254,340
7,亚利桑那,7.22,Arizona,True,0,0,148683,0,2918,174
8,马萨诸塞,7.22,Massachusetts,True,0,0,114033,0,8450,94347
9,宾夕法尼亚,7.22,Pennsylvania,True,0,0,107414,0,7046,1369


In [16]:
data_pair_list = [(i, int(j)) for i, j in zip(country_region_data['nameMap'], country_region_data['confirm'])]
(
    Map(init_opts=opts.InitOpts(theme=ThemeType.ROMANTIC))
    .add(
        maptype=country, # 使用具体国家地图
        series_name='累计确诊', 
        data_pair=data_pair_list, # data_pair是放数据的地方，类别和属性值
        label_opts=opts.LabelOpts(is_show=False),
        is_map_symbol_show=False,
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title=f"{country}{day} 累计确诊"),
        visualmap_opts=opts.VisualMapOpts(is_piecewise=True, pieces=pieces) # 使用自定义颜色范围
    )
).render(f'{filepath_visual}{country}{day} 累计确诊.html')

f:\python\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


'C:\\Users\\Jarvis\\Desktop\\Python实习\\PythonInternship\\Python爬虫学习\\部分可视化页面\\美国7.22 累计确诊.html'

In [17]:
data_pair_list = [(i, int(j)) for i, j in zip(country_region_data['nameMap'], country_region_data['dead'])]
(
    Map(init_opts=opts.InitOpts(theme=ThemeType.ROMANTIC))
    .add(
        maptype=country, # 使用具体国家地图
        series_name='累计死亡', 
        data_pair=data_pair_list, # data_pair是放数据的地方，类别和属性值
        label_opts=opts.LabelOpts(is_show=False),
        is_map_symbol_show=False,
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title=f"{country}{day} 累计死亡"),
        visualmap_opts=opts.VisualMapOpts(is_piecewise=True, pieces=pieces) # 使用自定义颜色范围
    )
).render(f'{filepath_visual}{country}{day} 累计死亡.html')

f:\python\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


'C:\\Users\\Jarvis\\Desktop\\Python实习\\PythonInternship\\Python爬虫学习\\部分可视化页面\\美国7.22 累计死亡.html'

In [18]:
data_pair_list = [(i, int(j)) for i, j in zip(country_region_data['nameMap'], country_region_data['heal'])]
(
    Map(init_opts=opts.InitOpts(theme=ThemeType.ROMANTIC))
    .add(
        maptype=country, # 使用具体国家地图
        series_name='累计治愈', 
        data_pair=data_pair_list, # data_pair是放数据的地方，类别和属性值
        label_opts=opts.LabelOpts(is_show=False),
        is_map_symbol_show=False,
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(title=f"{country}{day} 累计治愈"),
        visualmap_opts=opts.VisualMapOpts(is_piecewise=True, pieces=pieces) # 使用自定义颜色范围
    )
).render(f'{filepath_visual}{country}{day} 累计治愈.html')

f:\python\lib\site-packages\pyecharts\charts\chart.py:14: PendingDeprecationWarning: pyecharts 所有图表类型将在 v1.9.0 版本开始强制使用 ChartItem 进行数据项配置 :)
  super().__init__(init_opts=init_opts)


'C:\\Users\\Jarvis\\Desktop\\Python实习\\PythonInternship\\Python爬虫学习\\部分可视化页面\\美国7.22 累计治愈.html'